In [1]:
# importing required libraries
import re
import os
import time
import gensim
import requests
import numpy as np
import pandas as pd
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(2021)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\surya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_csv('combined-csv-files.csv')

C:\Users\surya\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (81) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_lda = pd.DataFrame()
df_lda['post_content'] = df['post_content'].values

In [4]:
# df_lda = df_lda.dropna()
np.where(pd.isnull(df_lda.post_content))
df_lda.fillna('No Comment', inplace=True)
df_lda = df_lda.reset_index(drop=True)

In [5]:
df_lda['index'] = df_lda.index
documents = df_lda
documents

,post_content,index
0,"Hey, so Iâ€™m experiencing a burning sensation...",0
1,Sounds like a panic attack. I had the same thi...,1
2,I really hope so. Every time I think I have it...,2
3,I tested POSITIVE for COVID19. I had something...,3
4,"Wow, I really wish you a safe and speedy recov...",4
...,...,...
39220,"Get the antibodies, I would get help sooner th...",39220
39221,"I'm late seeing this post, to the original pos...",39221
39222,91% isnâ€™t that bad. \nYou really shouldnâ€™t...,39222
39223,Thank you for your submission!\n\nPlease remem...,39223


In [6]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [7]:
doc_sample = documents[documents['index'] == 4].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Wow,', 'I', 'really', 'wish', 'you', 'a', 'safe', 'and', 'speedy', 'recovery.', 'How', 'long', 'have', 'you', 'been', 'positive?', 'Are', 'you', 'scared', 'that', 'you', 'might', 'need', 'to', 'go', 'to', 'the', 'ER?']


 tokenized and lemmatized document: 
['wish', 'safe', 'speedi', 'recoveri', 'long', 'posit', 'scar', 'need']


In [8]:
processed_docs = documents['post_content'].map(preprocess)
processed_docs

0        [experi, burn, sensat, right, middl, chest, co...
1        [sound, like, panic, attack, thing, begin, iso...
2        [hope, time, think, hold, breath, long, hold, ...
3        [test, posit, covid, similar, lung, headach, f...
4        [wish, safe, speedi, recoveri, long, posit, sc...
                               ...                        
39220               [antibodi, help, sooner, later, fight]
39221    [late, see, post, origin, poster, share, stori...
39222    [isnâ, shouldnâ, concern, reach, go, indic, yo...
39223    [thank, submiss, rememb, read, rule, ensur, po...
39224                                            [comment]
Name: post_content, Length: 39225, dtype: object

#### BOW

In [9]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 burn
1 chest
2 cough
3 covid
4 experi
5 havenâ
6 middl
7 overreact
8 relat
9 right
10 sensat


In [10]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

#### Gensim doc2bow

In [11]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4]

[(26, 1), (34, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1)]

In [12]:
bow_doc_4 = bow_corpus[4]
for i in range(len(bow_doc_4)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4[i][0], 
                                               dictionary[bow_doc_4[i][0]], 
bow_doc_4[i][1]))

Word 26 ("long") appears 1 time.
Word 34 ("posit") appears 1 time.
Word 40 ("need") appears 1 time.
Word 41 ("recoveri") appears 1 time.
Word 42 ("safe") appears 1 time.
Word 43 ("scar") appears 1 time.
Word 44 ("speedi") appears 1 time.
Word 45 ("wish") appears 1 time.


#### TF-IDF

In [13]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.29951182899564494),
 (1, 0.21307625924552023),
 (2, 0.1808139572961138),
 (3, 0.08913797316611903),
 (4, 0.19864173104101035),
 (5, 0.30057486181479814),
 (6, 0.36754224161804466),
 (7, 0.46602041636816205),
 (8, 0.2805825643439663),
 (9, 0.13259632023172582),
 (10, 0.34702545550302927),
 (11, 0.27137823820192175),
 (12, 0.19732614701792847),
 (13, 0.11343284853345226)]


#### LDA using BOW

In [14]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [15]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.058*"time" + 0.032*"thank" + 0.030*"comment" + 0.030*"hand" + 0.030*"rule" + 0.030*"question" + 0.030*"post" + 0.029*"kind" + 0.029*"read" + 0.029*"concern"
Topic: 1 
Words: 0.057*"test" + 0.027*"posit" + 0.020*"negat" + 0.013*"time" + 0.013*"virus" + 0.012*"day" + 0.012*"symptom" + 0.012*"go" + 0.012*"week" + 0.011*"work"
Topic: 2 
Words: 0.063*"https" + 0.052*"remov" + 0.046*"covid" + 0.034*"post" + 0.032*"reddit" + 0.020*"posit" + 0.020*"comment" + 0.017*"messag" + 0.015*"articl" + 0.014*"question"
Topic: 3 
Words: 0.063*"smell" + 0.044*"tast" + 0.019*"like" + 0.018*"lose" + 0.018*"covid" + 0.016*"sens" + 0.015*"thing" + 0.014*"symptom" + 0.011*"test" + 0.010*"posit"
Topic: 4 
Words: 0.045*"covid" + 0.039*"test" + 0.016*"posit" + 0.015*"symptom" + 0.013*"vaccin" + 0.012*"antibodi" + 0.010*"think" + 0.010*"infect" + 0.009*"peopl" + 0.009*"get"
Topic: 5 
Words: 0.040*"test" + 0.040*"symptom" + 0.026*"fever" + 0.024*"cough" + 0.023*"day" + 0.020*"feel" + 0.017*"throa

#### LDA using TF-IDF

In [16]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.145*"delet" + 0.114*"remov" + 0.069*"thank" + 0.013*"vaccin" + 0.008*"share" + 0.006*"better" + 0.006*"respons" + 0.005*"time" + 0.005*"trump" + 0.005*"covid"
Topic: 1 Word: 0.014*"thread" + 0.013*"chat" + 0.012*"situat" + 0.012*"allow" + 0.011*"experi" + 0.011*"current" + 0.010*"place" + 0.010*"glad" + 0.009*"post" + 0.009*"covid"
Topic: 2 Word: 0.006*"send" + 0.006*"symptom" + 0.006*"good" + 0.006*"peopl" + 0.005*"sorri" + 0.005*"feel" + 0.005*"know" + 0.005*"test" + 0.005*"day" + 0.005*"like"
Topic: 3 Word: 0.042*"time" + 0.035*"comment" + 0.030*"despair" + 0.030*"align" + 0.030*"automat" + 0.030*"perform" + 0.030*"purpos" + 0.030*"toler" + 0.030*"submiss" + 0.030*"ensur"
Topic: 4 Word: 0.008*"hope" + 0.008*"feel" + 0.007*"symptom" + 0.007*"better" + 0.007*"test" + 0.006*"know" + 0.006*"peopl" + 0.006*"long" + 0.005*"soon" + 0.005*"like"
Topic: 5 Word: 0.007*"like" + 0.007*"peopl" + 0.006*"mask" + 0.005*"feel" + 0.005*"covid" + 0.005*"know" + 0.005*"think" + 0.004*"

In [17]:
processed_docs[4]

['wish', 'safe', 'speedi', 'recoveri', 'long', 'posit', 'scar', 'need']

In [18]:
for index, score in sorted(lda_model[bow_corpus[4]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8999717831611633	 
Topic: 0.025*"peopl" + 0.019*"vaccin" + 0.016*"know" + 0.013*"thank" + 0.011*"covid" + 0.010*"mask" + 0.009*"think" + 0.009*"like" + 0.009*"work" + 0.008*"go"

Score: 0.011115719564259052	 
Topic: 0.057*"test" + 0.027*"posit" + 0.020*"negat" + 0.013*"time" + 0.013*"virus" + 0.012*"day" + 0.012*"symptom" + 0.012*"go" + 0.012*"week" + 0.011*"work"

Score: 0.011115282773971558	 
Topic: 0.058*"time" + 0.032*"thank" + 0.030*"comment" + 0.030*"hand" + 0.030*"rule" + 0.030*"question" + 0.030*"post" + 0.029*"kind" + 0.029*"read" + 0.029*"concern"

Score: 0.011114761233329773	 
Topic: 0.045*"covid" + 0.039*"test" + 0.016*"posit" + 0.015*"symptom" + 0.013*"vaccin" + 0.012*"antibodi" + 0.010*"think" + 0.010*"infect" + 0.009*"peopl" + 0.009*"get"

Score: 0.011114525608718395	 
Topic: 0.063*"https" + 0.052*"remov" + 0.046*"covid" + 0.034*"post" + 0.032*"reddit" + 0.020*"posit" + 0.020*"comment" + 0.017*"messag" + 0.015*"articl" + 0.014*"question"

Score: 0.0111144017428

In [19]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8999677300453186	 
Topic: 0.014*"thread" + 0.013*"chat" + 0.012*"situat" + 0.012*"allow" + 0.011*"experi" + 0.011*"current" + 0.010*"place" + 0.010*"glad" + 0.009*"post" + 0.009*"covid"

Score: 0.011117454618215561	 
Topic: 0.008*"hope" + 0.008*"feel" + 0.007*"symptom" + 0.007*"better" + 0.007*"test" + 0.006*"know" + 0.006*"peopl" + 0.006*"long" + 0.005*"soon" + 0.005*"like"

Score: 0.0111150573939085	 
Topic: 0.011*"youâ" + 0.011*"test" + 0.011*"thank" + 0.008*"think" + 0.008*"doctor" + 0.006*"vitamin" + 0.006*"know" + 0.006*"okay" + 0.006*"check" + 0.006*"tell"

Score: 0.011114881373941898	 
Topic: 0.006*"send" + 0.006*"symptom" + 0.006*"good" + 0.006*"peopl" + 0.005*"sorri" + 0.005*"feel" + 0.005*"know" + 0.005*"test" + 0.005*"day" + 0.005*"like"

Score: 0.011114859953522682	 
Topic: 0.030*"test" + 0.021*"sorri" + 0.016*"symptom" + 0.014*"negat" + 0.013*"day" + 0.012*"loss" + 0.011*"posit" + 0.011*"updat" + 0.010*"hear" + 0.007*"month"

Score: 0.011114725843071938	 
Topic:

In [20]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.8196550607681274	 Topic: 0.029*"pain" + 0.026*"feel" + 0.025*"like" + 0.019*"symptom" + 0.016*"go"
Score: 0.020045502111315727	 Topic: 0.045*"covid" + 0.039*"test" + 0.016*"posit" + 0.015*"symptom" + 0.013*"vaccin"
Score: 0.0200448427349329	 Topic: 0.025*"peopl" + 0.019*"vaccin" + 0.016*"know" + 0.013*"thank" + 0.011*"covid"
Score: 0.020043659955263138	 Topic: 0.063*"https" + 0.052*"remov" + 0.046*"covid" + 0.034*"post" + 0.032*"reddit"
Score: 0.020038286224007607	 Topic: 0.018*"feel" + 0.018*"better" + 0.015*"help" + 0.014*"like" + 0.013*"symptom"
Score: 0.02003522589802742	 Topic: 0.040*"test" + 0.040*"symptom" + 0.026*"fever" + 0.024*"cough" + 0.023*"day"
Score: 0.020034868270158768	 Topic: 0.018*"like" + 0.018*"feel" + 0.017*"breath" + 0.014*"go" + 0.013*"week"
Score: 0.020034784451127052	 Topic: 0.063*"smell" + 0.044*"tast" + 0.019*"like" + 0.018*"lose" + 0.018*"covid"
Score: 0.020034460350871086	 Topic: 0.057*"test" + 0.027*"posit" + 0.020*"negat" + 0.013*"time" + 0.013*

In [24]:
dtm_tfidf = np.zeros((39225,10), dtype=float)

row = 0
for doc in bow_corpus[0:39225]:
    sm = lda_model_tfidf.get_document_topics(doc)
    for e in sm:
        dtm_tfidf[row,e[0]] = e[1]
    row = row + 1
    
cp_tfidf = pd.DataFrame(dtm_tfidf)

cp_tfidf.columns = ['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10'] 
cp_tfidf.insert(0, 'post_content', documents.post_content)

cp_tfidf.to_csv("Doc_Out_TF-IDF.csv")

In [25]:
dtm_bow = np.zeros((39225,10), dtype=float)

row1 = 0
for doc1 in bow_corpus[0:39225]:
    sm1 = lda_model.get_document_topics(doc1)
    for e1 in sm1:
        dtm_bow[row1,e1[0]] = e1[1]
    row1 = row1 + 1
    
cp_bow = pd.DataFrame(dtm_bow)

cp_bow.columns = ['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10'] 
cp_bow.insert(0, 'post_content', documents.post_content)

cp_bow.to_csv("Doc_Out_BOW.csv")

In [ ]:
final_out = pd.DataFrame(columns=['Reference', 'Score', 'Index Values'])

In [ ]:
dict_out = {}
for k in range(1000):
    ordered_vocab = [(term, voc) for term, voc in lda_model_tfidf[bow_corpus[k]]]
    ordered_vocab = sorted(ordered_vocab, key=lambda tup: -1*tup[1])

    for index, score in ordered_vocab:
        dict_out.update({score: lda_model_tfidf.print_topic(index, 10)})
        
    temp = pd.DataFrame(dict_out.items(), columns=['Score', 'Index Values'])
    temp['Reference'] = k
    final_out = final_out.append(temp)
    del temp

### Note: "Index" column from "documents" dataframe maps to the "Reference" column in the "final_out" dataframe

#### Unused Code

In [ ]:
# df_lda['index'] = df_lda.index

# df_lda.drop('index', axis=1, inplace=True)
# df_lda.index.name
# df_lda.columns.name = 'index'

# ordered_vocab = [(term, voc.index, voc.count) for term, voc in model.wv.vocab.items()]
# ordered_vocab = sorted(ordered_vocab, key=lambda k: k[2])
# ordered_terms, term_indices, term_counts = zip(*ordered_vocab)
# word_vectors = pd.DataFrame(model.wv.syn0[term_indices, :], index=ordered_terms)

# ordered_vocab = [(term, voc) for term, voc in lda_model_tfidf[bow_corpus[10]]]
# ordered_vocab = sorted(ordered_vocab, key=lambda tup: -1*tup[1])

# df1 = pd.DataFrame([{score: lda_model_tfidf.print_topic(index, 10)} for index, score in ordered_vocab])

# df2 = pd.DataFrame(list(ordered_vocab))

# for i,k in dict_out.items():
#     print(i,k)

# df_out = pd.DataFrame(dict_out.items(), columns=['Score', 'Index Values'])

# dict_out = {}
# for index, score in ordered_vocab:
#     dict_out.update({score: lda_model_tfidf.print_topic(index, 10)})

# final_out.to_csv("Final_Out.csv")
# documents.to_csv("Post_Content.csv")